# Robótica Móvel - Trabalho Prático 1
---
**Nome Completo:** Daniel Terra Gomes

**Matrícula:** 2025702870 

**Programa:** Mestrando do PPGCC
**Departamento:** Departamento de Ciência da Computação
**Instituição:** ICEx-UFMG
**E-mail:** danielterragomes@ufmg.br

**Data:** 11 de setembro de 2025

---

## Exercício 1: Criação da Cena no CoppeliaSim

O primeiro passo foi criar uma cena no CoppeliaSim contendo um robô móvel (Pioneer 3DX) e cinco outros objetos distintos (pessoa, mesa, cadeira, planta, e caixote) para popular o ambiente de simulação. A imagem abaixo documenta a cena criada.

![CoppeliaSim Scene](Screenshot%20from%202025-09-04%2009-26-53.png)

*Figura 1: Cena criada no CoppeliaSim contendo o robô Pioneer 3DX e cinco objetos distintos (pessoa, mesa, cadeira, planta, e caixote)*

## Exercício 2: Diagrama de Transformações

O diagrama a seguir representa as relações entre os sistemas de coordenadas dos objetos na cena. O frame do Mundo {W} serve como referência global. 

![Transformation Diagram](ex2-diagram.drawio.png)

*Figura 2: Diagrama de transformações mostrando o sistema de coordenadas do ex.1 e as relações entre os diferentes sistemas de coordenadas na cena. As setas sólidas (Verdes) representam transformações diretas do objeto conhecidos, enquanto a seta (Vermelha) mostra uma transformação desejada (Desconhecida).*

## Exercício 3: Código - Matrizes de Transformação Homogênea

O código abaixo conecta-se à simulação, obtém o pose (posição e orientação) de cada objeto em relação ao frame do mundo {W} e calcula a matriz de transformação homogênea correspondente.

In [ ]:
import numpy as np
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import math

# --- Funções Auxiliares ---
def euler_to_rotation_matrix(alpha, beta, gamma):
    """Converte ângulos de Euler (em radianos) para uma matriz de rotação 3x3."""
    # TODO: Implementar a construção das matrizes Rx, Ry, Rz
    R_x = np.array([[1, 0, 0],
                      [0, math.cos(alpha), -math.sin(alpha)],
                      [0, math.sin(alpha), math.cos(alpha)]])

    R_y = np.array([[math.cos(beta), 0, math.sin(beta)],
                      [0, 1, 0],
                      [-math.sin(beta), 0, math.cos(beta)]])

    R_z = np.array([[math.cos(gamma), -math.sin(gamma), 0],
                      [math.sin(gamma), math.cos(gamma), 0],
                      [0, 0, 1]])

    # TODO: Multiplicar as matrizes na ordem correta (Rz * Ry * Rx)
    R = np.dot(R_z, np.dot(R_y, R_x))
    return R

def create_homogeneous_matrix(R, P):
    """Monta a matriz de transformação homogênea 4x4 a partir de R (3x3) e P (3x1)."""
    T = np.identity(4)
    T[:3, :3] = R
    T[:3, 3] = P
    return T

# --- Conexão com o CoppeliaSim ---
client = RemoteAPIClient()
sim = client.require('sim')
sim.startSimulation()

print('Conectado ao CoppeliaSim!')

# --- Lógica Principal ---
object_paths = {
    'Robot': '/PioneerP3DX',
    'Bill': '/Bill',
    'Chair': '/DiningChair',
    'Table': '/DiningTable',
    'Plant': '/IndoorPlant',
    'Crate': '/Crate'
}

transforms = {}

for name, path in object_paths.items():
    print(f"\n--- Processando: {name} ---")

    # 1. Obter o handle do objeto
    handle = sim.getObject(path)
    if handle == -1:
        print(f"Erro: Objeto '{name}' não encontrado no caminho '{path}'.")
        continue

    # 2. Obter posição e orientação em relação ao mundo
    position = sim.getObjectPosition(handle, sim.handle_world)
    orientation_euler = sim.getObjectOrientation(handle, sim.handle_world) # Em radianos!

    # 3. Calcular a Matriz de Rotação
    rotation_matrix = euler_to_rotation_matrix(orientation_euler[0], orientation_euler[1], orientation_euler[2])

    # 4. Montar a Matriz Homogênea
    homogeneous_matrix = create_homogeneous_matrix(rotation_matrix, position)
    transforms[name] = homogeneous_matrix

    # 5. Imprimir os resultados
    print(f"Posição (x, y, z): {np.round(position, 3)}")
    print(f"Orientação (alpha, beta, gamma) em graus: {np.round(np.rad2deg(orientation_euler), 3)}")
    print(f"Matriz de Transformação Homogênea de '{name}' para o Mundo:")
    print(np.round(homogeneous_matrix, 3))

sim.stopSimulation()

## Exercício 4: Transformação Inversa

Calcular a transformação do mundo em relação a um objeto (e.g., ${}_{W}^{R}T$). Isso é feito calculando a inversa da matriz ${}_{R}^{W}T$.

In [ ]:
# TODO: Pegar uma das matrizes calculadas (ex: a do robô)
T_Robot_W = transforms.get('Robot')

if T_Robot_W is not None:
    # TODO: Calcular a inversa usando np.linalg.inv()
    T_W_Robot = np.linalg.inv(T_Robot_W)

    print("Matriz de Transformação do Robô para o Mundo (T_R^W):")
    print(np.round(T_Robot_W, 3))
    print("\nMatriz de Transformação do Mundo para o Robô (T_W^R):")
    print(np.round(T_W_Robot, 3))

## Exercício 5: Transformação Composta

Calcular a transformação de um objeto para outro (e.g., a planta em relação ao robô, ${}_{P}^{R}T$). A fórmula é: ${}_{P}^{R}T = ({}_{R}^{W}T)^{-1} \cdot {}_{P}^{W}T$.

In [ ]:
# TODO: Pegar as matrizes necessárias
T_Robot_W = transforms.get('Robot')
T_Plant_W = transforms.get('Plant')

if T_Robot_W is not None and T_Plant_W is not None:
    # TODO: Calcular a inversa de T_Robot_W
    T_W_Robot = np.linalg.inv(T_Robot_W)

    # TODO: Multiplicar as matrizes para obter a transformação composta
    T_Plant_Robot = np.dot(T_W_Robot, T_Plant_W)

    print("Matriz de Transformação da Planta para o Robô (T_P^R):")
    print(np.round(T_Plant_Robot, 3))

## Exercício 6: Mapeamento de Pontos

Definir um ponto no referencial de um objeto (e.g., a planta) e usar a matriz de transformação para descobrir onde este ponto se localiza no referencial do robô.

In [ ]:
# TODO: Definir um ponto no referencial da Planta {P}
# Lembre-se de usar coordenadas homogêneas (adicionar o 1 no final)
P_point_in_Plant_frame = np.array([0.1, 0, 0.5, 1]) # Ex: um ponto 10cm à frente e 50cm acima da origem da planta

# TODO: Usar a matriz T_Plant_Robot para transformar o ponto
if 'T_Plant_Robot' in locals():
    P_point_in_Robot_frame = np.dot(T_Plant_Robot, P_point_in_Plant_frame)

    print(f"O ponto {P_point_in_Plant_frame[:3]} no referencial da Planta")
    print(f"está na posição {np.round(P_point_in_Robot_frame[:3], 3)} no referencial do Robô.")

## Detalhes da Implementação

*(Nesta seção, descreva as decisões que você tomou, as bibliotecas que usou (NumPy, ZMQ, etc.), e como o seu código está estruturado. Explique o que cada função auxiliar faz.)*

## Conclusão

*(Aqui você pode fazer comentários gerais sobre o trabalho, as principais dificuldades que encontrou, e o que aprendeu com a atividade.)*